In [4]:
import pandas as pd
import numpy as np
from netalign.approx_isorank.io_utils import compute_adjacency, compute_pairs
from netalign.approx_isorank.isorank_compute import compute_isorank, compute_greedy_assignment, pair_acc
from netalign.approx_isorank.pair_evaluations import compute_edge_correctness, semantic_sim, symmetric_substructure, lccs
from numpy.linalg import norm
import json
from scipy.io import loadmat

In [5]:
sp1 = "bakers"
sp2 = "rat"

finalloc = f"temp-data/{sp1}-{sp2}.mat"

MAT = loadmat(finalloc)[f"{sp1}_{sp2}"]

In [6]:
pairs = compute_greedy_assignment(MAT, 100)

In [7]:
dataloc = "../../data/intact/"
df1 = pd.read_csv(f"{dataloc}/{sp1}.s.tsv", header = None, sep = "\t")
df2 = pd.read_csv(f"{dataloc}/{sp2}.s.tsv", header = None, sep = "\t")

df1

,0,1
0,P08539,P18851
1,P11433,P18852
2,P11433,P21268
3,P11433,P18851
4,P18851,P18852
...,...,...
109681,Q07821,Q12425
109682,P47158,Q12425
109683,P47158,Q07821
109684,P47158,Q12425


In [8]:
with open(f"temp-data/{sp1}.json") as ij1, open(f"temp-data/{sp2}.json") as ij2:
    nm1 = json.load(ij1)
    nm2 = json.load(ij2)

dfa1 = df1.copy()
dfa2 = df2.copy()

dfa1[0] = dfa1[0].apply(lambda x : nm1[x])
dfa1[1] = dfa1[1].apply(lambda x : nm1[x])


dfa2[0] = dfa2[0].apply(lambda x : nm2[x])
dfa2[1] = dfa2[1].apply(lambda x : nm2[x])

pairs = [(i, j[0, 0]) for i, j in pairs]

In [12]:

rnA1 = {v: k for k, v in nm1.items()}
rnA2 = {v: k for k, v in nm2.items()}

annot1 = f"../../data/go/{sp1}.output.mapping.gaf"
annot2 = f"../../data/go/{sp2}.output.mapping.gaf"
FC={}
pairs_ = [(rnA1[p], rnA2[q]) for p, q in pairs]

for gotype in ["molecular_function", "biological_process", "cellular_component"]:
    fc = semantic_sim(pairs_, df1, df2,
                    obofile = "../../data/go/go-basic.obo",
                    annot1file=annot1,
                    annot2file = annot2,
                    type = gotype)
    print(f"\t For GO={gotype}, FC = {fc}")
    FC[f"FC-{gotype}(FINAL)"] = fc

  EXISTS: ../../data/go/go-basic.obo
../../data/go/go-basic.obo: fmt(1.2) rel(2022-12-04) 46,763 Terms; optional_attrs(relationship)
	 For GO=molecular_function, FC = 0.504709526965961
  EXISTS: ../../data/go/go-basic.obo
../../data/go/go-basic.obo: fmt(1.2) rel(2022-12-04) 46,763 Terms; optional_attrs(relationship)
	 For GO=biological_process, FC = 0.23684912300624475
  EXISTS: ../../data/go/go-basic.obo
../../data/go/go-basic.obo: fmt(1.2) rel(2022-12-04) 46,763 Terms; optional_attrs(relationship)
	 For GO=cellular_component, FC = 0.40140898061509234
